In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# This is a simple exploration of the train label 

* Train a simple Word2Vec model for 2 dimensions and get the average embeddings of each the labels
* Cluster them by DBSCAN
* Visualize each cluster using scatter plot

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from sklearn.cluster import KMeans, DBSCAN
import plotly.express as px
import seaborn as sns

import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

import plotly.offline as pyo
import plotly.express as px


In [ ]:
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train_df

# Form corpus

In [ ]:
# Frame the corpus which is the unique train labels

train_unique_label = np.unique(train_df.cleaned_label.values.astype('str'))
corpus = []
for label in train_unique_label:
    corpus.append(label.split( ))

# Word2Vec Train function

In [ ]:
def train_w2vec(corpus, dim, epochs):
    '''
    Function to train a simple word2Vec model
    '''
    model = word2vec.Word2Vec(vector_size = dim, min_count= 1)
    model.build_vocab(corpus)
    model.train(corpus, epochs=epochs, total_examples=model.corpus_count)
    
    return model

# Function to Get average embeddings

In [ ]:
def get_avg_label_embedding(corpus, dim, model):
    '''
    Function to get the average word embeddings when the label is more than 1 word
    '''

    label_embedding = []
    for label in corpus:
        embedding_sum=np.zeros(shape=(dim, ))
        for tok in label:
            temp = model.wv[tok].copy()
            
            embedding_sum+=temp
            
        avg_embedding = embedding_sum/len(label)
        label_embedding.append(avg_embedding.tolist())
    
    label_embedding = np.array(label_embedding)
    
    return label_embedding


# Train and get embedding

In [ ]:
# Train the word2Vec model for few epochs
dim = 2
model = train_w2vec(corpus, dim, epochs=100)
label_embedding = get_avg_label_embedding(corpus, dim, model)

label_embedding.shape

# Cluster the data using DBSCAN

In [ ]:
DBSCAN_cluster = DBSCAN(min_samples=1, eps=0.08)
DBSCAN_cluster.fit(label_embedding)
clusters = DBSCAN_cluster.labels_

print(f"Total clusters formed are : {len(clusters)}")

# Visualize
-> However over the points for name of the dataset

In [ ]:
# Set notebook mode to work in offline
pyo.init_notebook_mode()

# Create traces
trace0 = px.scatter(x = label_embedding[:, 0], 
                    y = label_embedding[:, 1], 
                    color = clusters, 
                    hover_name=train_unique_label
)

trace0.show()

With a quick hover on the points it looks like the cluster are good and similar domains are grouped together with few data points which are not clustered right. However you can play around with it by trying

* Different epochs for Word2Vec, 
* Train for more Dimensions and reduce them using t-sne or PCA
* Play with different cluster distance mertrics for DBSCAN clusters or different cluster algorithms